In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            
            full_data_rows_list.append(line) 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

## Dataset contains the following columns: 
[0] artist <br>
[1] firstName of user <br>
[2] gender of user <br>
[3] item number in session <br>
[4] last name of user <br>
[5] length of the song <br>
[6] level (paid or free song) <br>
[7] location of the user <br>
[8] sessionId <br>
[9] song title <br>
[10] userId <br>

<img src="images/image_event_datafile_new.jpg">

In [4]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

In [5]:
try:
    session.execute("CREATE KEYSPACE IF NOT EXISTS sparkify \
    WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}")
except Exceptions as e:
    print(e) 

In [6]:
try:
    session.set_keyspace('sparkify')
except Exceptions as e:
    print(e)

# Goal 1. Get the song information of specific item in specific session.
**Example query:** `SELECT artist, song, length FROM session_library WHERE sessionId = 338 AND itemInSession = 4` 

Because the combination of `sessionId` and `itemInSession` can make each data unique and will be used as criteria, they have to be in Primary Key; And using either `sessionId` or `itemInSession` as partition key can evenly distribute data.

In [7]:
########## drop and create the table ##########
drop_session_library = "DROP TABLE IF EXISTS session_library"

create_session_library = "CREATE TABLE IF NOT EXISTS session_library (sessionId INT, iteminsession INT, \
artists TEXT, length FLOAT, song TEXT, userId INT, PRIMARY KEY (sessionId, itemInsession))"

try:
    session.execute(drop_session_library)
    session.execute(create_session_library)
    
except Exception as e:
    print(e)

########## insert values ##########
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_library (sessionId, iteminsession, artists, length, song, userId)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9], int(line[10])))

########## select value to see result ##########  

select_session = "SELECT artists, song, length FROM session_library WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(select_session)
except Exception as e:
    print(e)
for row in rows:
    print(row.artists, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Goal 2. Get songs information order by listening sequence for a specific user during a specific session.
**Example query:** `SELECT artist, song, firstName, lastName FROM user_library WHERE userId = 10 AND sessionId =182` 

To achieve `WHERE` clause filtering purpose, we need `userId` and `sessionId` in the Primary Key. But the combination of `userId` and `sessionId` can not make each data unique, so `itemInSession` is also needed. To list data in the order of `itemInSession`, `itemInSession` needs to be clustering column. To avoid error, Use *COMPOSITE PARTITION KEY* for `userId` and `sessionId`.

In [19]:
########## drop and create the table ##########
drop_user_library = "DROP TABLE IF EXISTS user_library"

create_user_library = "CREATE TABLE IF NOT EXISTS user_library (userId INT, sessionId INT, itemInsession INT, artist TEXT, song TEXT, \
firstName TEXT, lastName TEXT, PRIMARY KEY ((userId, sessionId),itemInsession))"

try:
    session.execute(drop_user_library)
    session.execute(create_user_library)
except Exception as e:
    print(e)

########## insert values ##########

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library (userId, sessionId, itemInsession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))
        
########## select value to see result ##########  
select_user = "SELECT artist, song, firstName, lastName FROM user_library WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(select_user)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Goal 3. Who listen to specific song.
**Example query:** `SELECT firstName, lastName FROM song_library WHERE song = 'All Hands Against His Own'` 

In [20]:
## Query 3: every user name (first and last) in music app history who listened to the song 'All Hands Against His Own'

########## drop and create the table ##########
drop_song_library = "DROP TABLE IF EXISTS song_library"

create_song_library = "CREATE TABLE IF NOT EXISTS song_library (song TEXT, firstName TEXT, lastName TEXT,userId INT, PRIMARY KEY (song, userId))"

try:
    session.execute(drop_song_library)
    session.execute(create_song_library)
except Exception as e:
    print(e)

########## insert values ##########

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (song, firstName, lastName, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        session.execute(query, (line[9],line[1],line[4],int(line[10])))
        
########## select  ##########  
select_song = "SELECT firstName, lastName FROM song_library WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(select_song)
except Exception as e:
    print(e)
for row in rows:
    print(row.firstname, row.lastname)   

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [21]:
try:
    session.execute(drop_session_library)
    session.execute(drop_song_library)
    session.execute(drop_user_library)
except Exception as e:
    print(e)

In [22]:
session.shutdown()
cluster.shutdown()